In [1]:
import os
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Consignment-Pricing-Prediction'

In [230]:
import pandas as pd

df = pd.read_csv("artifacts\data_ingestion\SCMS_Delivery_History_Dataset.csv", usecols=[
    'Project Code', 'PQ #', 'Country', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'PQ First Sent to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'Weight (Kilograms)', 'Freight Cost (USD)', 'Line Item Insurance (USD)','Scheduled Delivery Date', 'Delivered to Client Date'])

# df = pd.read_csv('artifacts/data_ingestion/SCMS_Delivery_History_Dataset.csv', index_col=['ID', 'PO / SO #', 'ASN/DN #', 'Managed By', 'PO Sent to Vendor Date',
#        'First Line Designation'])

In [216]:
df.columns 

Index(['Project Code', 'PQ #', 'Country', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'PQ First Sent to Client Date',
       'Scheduled Delivery Date', 'Delivered to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)'],
      dtype='object')

In [217]:
df.head()

,Project Code,PQ #,Country,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,Scheduled Delivery Date,Delivered to Client Date,Delivery Recorded Date,...,Dosage Form,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,100-CI-T01,Pre-PQ Process,Côte d'Ivoire,Direct Drop,EXW,Air,Pre-PQ Process,2-Jun-06,2-Jun-06,2-Jun-06,...,Test kit,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,13,780.34,NaN
1,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,14-Nov-06,14-Nov-06,14-Nov-06,...,Oral suspension,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",358,4521.5,NaN
2,100-CI-T01,Pre-PQ Process,Côte d'Ivoire,Direct Drop,FCA,Air,Pre-PQ Process,27-Aug-06,27-Aug-06,27-Aug-06,...,Test kit,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,171,1653.78,NaN
3,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,1-Sep-06,1-Sep-06,1-Sep-06,...,Tablet,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",1855,16007.06,NaN
4,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,11-Aug-06,11-Aug-06,11-Aug-06,...,Capsule,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",7590,45450.08,NaN


In [218]:
df.shape

(10324, 27)

In [219]:
# df =df.dropna()

In [220]:
type(df)

pandas.core.frame.DataFrame

In [221]:
import numpy as np
test_arr = pd.DataFrame(np.arange(10324))
test_arr

,0
0,0
1,1
2,2
3,3
4,4
...,...
10319,10319
10320,10320
10321,10321
10322,10322


In [222]:
df.columns

Index(['Project Code', 'PQ #', 'Country', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'PQ First Sent to Client Date',
       'Scheduled Delivery Date', 'Delivered to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)'],
      dtype='object')

In [223]:
to_drop= []
for i in df['Freight Cost (USD)']:
    try:
        float(i)
    except:
        to_drop.append(i)

In [224]:
df.shape

(10324, 27)

In [239]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, random_state=42, test_size=0.2)
train.shape, test.shape

((3656, 27), (915, 27))

pandas.core.frame.DataFrame

In [233]:
from sklearn.pipeline import FunctionTransformer, make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

def transform_dataset(testDF: pd.DataFrame):
    # Droping the na values from the dataframe
    testDF=testDF.dropna()

    # Checking the object values in the Weight Column
    to_drop_weight= []
    for i in testDF['Weight (Kilograms)']:
        try:
            float(i)
        except:
            to_drop_weight.append(i)
    # Dropping the object values from the Weight Column
    testDF = testDF[testDF['Weight (Kilograms)'].isin(to_drop_weight)==False]

    # Checking the object values in the Freight Column
    to_drop= []
    for i in testDF['Freight Cost (USD)']:
        try:
            float(i)
        except:
            to_drop.append(i)        
    # Dropping the object values from the Freight Column
    testDF = testDF[testDF['Freight Cost (USD)'].isin(to_drop)==False]

    # Splitting the scheduled Delivery date column to three respective columns storing the date, month and year
    testDF['Scheduled_Delivery_Date_D'] = [i.split('-')[0] for i in testDF['Scheduled Delivery Date']]
    testDF['Scheduled_Delivery_Date_M'] = [i.split('-')[1] for i in testDF['Scheduled Delivery Date']]
    testDF['Scheduled_Delivery_Date_Y'] = [i.split('-')[2] for i in testDF['Scheduled Delivery Date']]

    # Splitting the delivered to client date column to three respective columns storing the date, month and year
    testDF['Delivered_to_Client_Date_D'] = [i.split('-')[0] for i in testDF['Delivered to Client Date']]
    testDF['Delivered_to_Client_Date_M'] = [i.split('-')[1] for i in testDF['Delivered to Client Date']]
    testDF['Delivered_to_Client_Date_Y'] = [i.split('-')[2] for i in testDF['Delivered to Client Date']]

    # Dropping the scheduled delivery date column and the client date column
    testDF = testDF.drop(columns=['Delivered to Client Date', 'Scheduled Delivery Date'], axis= 1)

    # Changing the datatype of the columns
    testDF['Weight (Kilograms)'] = testDF['Weight (Kilograms)'].astype(float)
    testDF['Freight Cost (USD)'] = testDF['Freight Cost (USD)'].astype(float)
    testDF['Scheduled_Delivery_Date_D'] = testDF['Scheduled_Delivery_Date_D'].astype(float)
    testDF['Scheduled_Delivery_Date_Y'] = testDF['Scheduled_Delivery_Date_Y'].astype(float)
    testDF['Delivered_to_Client_Date_D'] = testDF['Delivered_to_Client_Date_D'].astype(float)
    testDF['Delivered_to_Client_Date_Y'] = testDF['Delivered_to_Client_Date_Y'].astype(float)

    # Encoding the categorical columns
    label_encoder = LabelEncoder()
    testDF = testDF.apply(label_encoder.fit_transform)

    # # Splitting the dataset
    # y = testDF['Freight Cost (USD)']
    # x = testDF.drop('Freight Cost (USD)', axis=1)

    # x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

    
    # stdscal = StandardScaler()
    # stdscal.fit_transform(testDF)
    # testDF = stdscal
    return testDF

pipeline = make_pipeline(
    FunctionTransformer(transform_dataset),
)

In [234]:
df = pipeline.transform(df)

In [235]:
df.shape

(4571, 27)

In [236]:
import pickle
obj = pipeline
with open('ptahia.pkl', 'wb') as file:
            pickle.dump(obj, file)


In [237]:

import pickle 
loaded_preprocessor = None
with open('ptahia.pkl', 'rb') as file:
    loaded_preprocessor =pickle.load(file)
df = loaded_preprocessor.transform(df)

In [238]:
df.shape

(4571, 27)

In [109]:
test = {
    'df': [1,11,1,11,1,1,1,1,1],
    'df2': [2,22,2,22,2,2,np.NAN,2,2],
    'df3': [3,33,3,33,3,np.NAN,3,3,3],
    'df4': [4,44,4,np.NAN,4,4,4,4,4],
}
test_df = pd.DataFrame(test)
test_df
# test_df.dropna(inplace=True)

,df,df2,df3,df4
0,1,2.0,3.0,4.0
1,11,22.0,33.0,44.0
2,1,2.0,3.0,4.0
3,11,22.0,33.0,NaN
4,1,2.0,3.0,4.0
5,1,2.0,NaN,4.0
6,1,NaN,3.0,4.0
7,1,2.0,3.0,4.0
8,1,2.0,3.0,4.0


In [110]:
pipeline.transform(test_df)
test_df

,df,df2,df3,df4
0,1,2.0,3.0,4.0
1,11,22.0,33.0,44.0
2,1,2.0,3.0,4.0
4,1,2.0,3.0,4.0
7,1,2.0,3.0,4.0
8,1,2.0,3.0,4.0


NoneType

In [94]:
test_arr.shape 

(10324, 1)

In [92]:
df.shape

(8158, 27)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6198 entries, 0 to 10316
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Project Code                  6198 non-null   object 
 1   PQ #                          6198 non-null   object 
 2   Country                       6198 non-null   object 
 3   Fulfill Via                   6198 non-null   object 
 4   Vendor INCO Term              6198 non-null   object 
 5   Shipment Mode                 5987 non-null   object 
 6   PQ First Sent to Client Date  6198 non-null   object 
 7   Scheduled Delivery Date       6198 non-null   object 
 8   Delivered to Client Date      6198 non-null   object 
 9   Delivery Recorded Date        6198 non-null   object 
 10  Product Group                 6198 non-null   object 
 11  Sub Classification            6198 non-null   object 
 12  Vendor                        6198 non-null   object 
 13  Item De

,0
0,0
1,1
2,2
3,3
4,4
...,...
10319,10319
10320,10320
10321,10321
10322,10322


In [6]:
df.insert(0,'test',test_arr)
# df.drop('test', inplace=True)
df.columns

Index(['test', 'ID', 'Project Code', 'PQ #', 'PO / SO #', 'ASN/DN #',
       'Country', 'Managed By', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'PQ First Sent to Client Date',
       'PO Sent to Vendor Date', 'Scheduled Delivery Date',
       'Delivered to Client Date', 'Delivery Recorded Date', 'Product Group',
       'Sub Classification', 'Vendor', 'Item Description',
       'Molecule/Test Type', 'Brand', 'Dosage', 'Dosage Form',
       'Unit of Measure (Per Pack)', 'Line Item Quantity', 'Line Item Value',
       'Pack Price', 'Unit Price', 'Manufacturing Site',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)'],
      dtype='object')

In [9]:
df.drop('test', inplace=True, axis=1)
df.columns

Index(['ID', 'Project Code', 'PQ #', 'PO / SO #', 'ASN/DN #', 'Country',
       'Managed By', 'Fulfill Via', 'Vendor INCO Term', 'Shipment Mode',
       'PQ First Sent to Client Date', 'PO Sent to Vendor Date',
       'Scheduled Delivery Date', 'Delivered to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'First Line Designation', 'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)'],
      dtype='object')

In [ ]:
df = pipeline.transform(df)

KeyError: "['ID', 'PO / SO #', 'ASN/DN #'] not found in axis"

In [ ]:
df.head()

,Project Code,PQ #,Country,Managed By,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,PO Sent to Vendor Date,Scheduled Delivery Date,...,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,First Line Designation,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
21,104-CI-T01,Pre-PQ Process,Côte d'Ivoire,PMO - US,Direct Drop,CIP,Air,Pre-PQ Process,7/13/07,2-Oct-07,...,60,5513,140581.5,25.50,0.42,"BMS Meymac, France",Yes,2126,Freight Included in Commodity Cost,224.93
22,108-VN-T01,Pre-PQ Process,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,7/4/07,15-Oct-07,...,240,1000,1920.0,1.92,0.01,"Aurobindo Unit III, India",Yes,941,4193.49,3.07
23,108-VN-T01,Pre-PQ Process,Vietnam,PMO - US,Direct Drop,FCA,Air,Pre-PQ Process,7/4/07,27-Aug-07,...,120,500,41095.0,82.19,0.68,ABBVIE Ludwigshafen Germany,Yes,117,1767.38,65.75
29,117-ET-T01,Pre-PQ Process,Ethiopia,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,10/3/07,20-Nov-07,...,60,64000,99200.0,1.55,0.03,"Aurobindo Unit III, India",Yes,4228,12237.61,158.72
32,108-VN-T01,Pre-PQ Process,Vietnam,PMO - US,Direct Drop,EXW,Air,Pre-PQ Process,11/19/07,21-Jan-08,...,30,1000,17000.0,17.00,0.57,Gilead(Nycomed) Oranienburg DE,Yes,76,2282.57,27.20


In [ ]:
df.shape

(8158, 30)

In [41]:
import os

In [1]:
df.head()

NameError: name 'df' is not defined